# Entrenamiento del modelo

In [ ]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [ ]:

# Definir transformaciones para los conjuntos de datos
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Cargar los conjuntos de datos
data_dir = 'directorio_datos'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Configurar el modelo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnext50_32x4d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # Adaptar a 5 clases
model = model.to(device)

# Definir el criterio de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Aquí añadirías el bucle de entrenamiento y validación


In [ ]:
def train_model(model, criterion, optimizer, num_epochs=12):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Cada época tiene una fase de entrenamiento y validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Establecer el modelo en modo de entrenamiento
            else:
                model.eval()   # Establecer el modelo en modo de evaluación

            running_loss = 0.0
            running_corrects = 0

            # Iterar sobre los datos.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Reiniciar los gradientes del optimizador
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estadísticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Entrenar el modelo
model_trained = train_model(model, criterion, optimizer, num_epochs=12)


In [ ]:
torch.save(model_trained.state_dict(), 'model_resnext.pth')


# Cargar el modelo

In [ ]:
# Cargar el modelo
model.load_state_dict(torch.load('/kaggle/working/model_resnext.pth'))
model.eval()  # Establecer en modo de evaluación

# Aquí debes cargar tus imágenes de prueba, aplicar las transformaciones necesarias y pasarlas al modelo


# Matriz de confusion y Recall

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, recall_score
import numpy as np
import os

# Asumiendo que 'model' ya está definido y cargado como se mostró anteriormente

# Transformaciones para las imágenes de prueba
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargar las imágenes de prueba
test_dir = 'directorio_datos_test'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)
class_names = test_dataset.classes

# Función para realizar predicciones y calcular la matriz de confusión
def predict_and_confusion_matrix(model, dataloader):
    y_pred = []
    y_true = []
    model.eval()  # Establecer el modelo en modo de evaluación
    with torch.no_grad():  # No necesitamos calcular gradientes
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    
    # Calcular la matriz de confusión
    conf_mat = confusion_matrix(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    return conf_mat, recall

# Calcular y mostrar la matriz de confusión
conf_mat, recall = predict_and_confusion_matrix(model, test_loader)
print(conf_mat)
print(f'Recall: {recall}')